In [1]:
import os
%pwd

'/config/workspace/experiments'

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [4]:
from ml_ops.constants import *
from ml_ops.utils.common import read_yaml, create_directories, save_json

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
        )

In [6]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [7]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.test_data_path]]

        predicted_qualities = model.predict(x_test)
        (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

        # Saving metrics as local
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path = Path(self.config.metric_file_name), data=scores)

In [8]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise e

[2024-02-02 14:39:19,954: INFO: common: yaml file: config/config.yaml loaded succesfullly] 
[2024-02-02 14:39:19,957: INFO: common: yaml file: params.yaml loaded succesfullly] 
[2024-02-02 14:39:19,960: INFO: common: yaml file: schema.yaml loaded succesfullly] 
[2024-02-02 14:39:19,962: INFO: common: created directory at: artifacts] 
[2024-02-02 14:39:19,963: INFO: common: created directory at: artifacts/model_evaluation] 


AttributeError: 'NoneType' object has no attribute 'test_data_path'